In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import os

In [2]:
model = YOLO("yolov8n-seg.pt")

In [3]:
input_root = "16_frame_clips_video"
output_root = "16_frame_clips_blurred"

# Ensure output root exists
os.makedirs(output_root, exist_ok=True)

In [5]:
def process_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create output video writer
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, verbose= False)
        mask_overlay = np.zeros(frame.shape, dtype=np.uint8)  # Create a blank mask for blurring

        for r in results:
            if r.masks is None:
                continue  # Skip if no masks found

            for box, mask in zip(r.boxes, r.masks.xy):
                cls = int(box.cls[0])  # Get class ID
                if cls == 0:  # Person class ID = 0
                    mask = np.array(mask, np.int32)  # Convert mask to NumPy array
                    
                    # Ensure mask is inside frame bounds
                    mask = np.clip(mask, [0, 0], [frame.shape[1] - 1, frame.shape[0] - 1])

                    # Fill mask overlay with white in the detected region
                    cv2.fillPoly(mask_overlay, [mask], (255, 255, 255))

        # Apply blur only where mask is white
        blurred = cv2.GaussianBlur(frame, (99, 99), 30)
        frame = np.where(mask_overlay > 0, blurred, frame)  # Blend original and blurred image

        out.write(frame)

    cap.release()
    out.release()

In [6]:
for root, dirs, files in os.walk(input_root):
    for file in files:
        if file.endswith(".mp4"):  # Process only .mp4 videos
            input_video_path = os.path.join(root, file)
            
            # Construct output path by replacing input root with output root
            relative_path = os.path.relpath(input_video_path, input_root)
            output_video_path = os.path.join(output_root, relative_path)

            print(f"Processing: {input_video_path} → {output_video_path}")
            process_video(input_video_path, output_video_path)

print("Blurring completed for all videos!")

Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_484.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_484.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_385.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_385.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_386.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_386.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_271.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_271.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_44.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_44.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_10.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_10.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_255.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_255.mp4
Processing: 16_frame_clips_video/sid2.MP4_16_clips/clip_599.mp4 → 16_frame_clips_blurred/sid2.MP4_16_clips/clip_599.mp4
Processing: 16_frame_clips_video/sid2.MP4_16